In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import os

In [3]:
pip install xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 1.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

# Charger le fichier XLS
file_path_xls = '/Users/pauline/Downloads/Ner_Derni_partie.xls'
xls_data = pd.read_excel(file_path_xls, engine='xlrd')

# Définir la fonction pour extraire les informations du fichier XML-TEI
def extract_xml_info(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
    
    long_title_element = root.find('.//tei:title[@type="titulo_completo"]', ns)
    short_title_element = root.find('.//tei:title[@type="titulo_breve"]', ns)
    pub_place = root.find('.//tei:pubPlace', ns)
    publisher = root.find('.//tei:publisher', ns)
    date = root.find('.//tei:date', ns)
    type_text = root.find('.//tei:term[@type="tipo_texto"]', ns)
    genre = root.find('.//tei:term[@type="sagrado_profano"]', ns)
    
    long_title = ''.join(long_title_element.itertext()).replace('\n', ' ').replace('  ', ' ') if long_title_element is not None else ''
    short_title = ''.join(short_title_element.itertext()).replace('\n', ' ').replace('  ', ' ') if short_title_element is not None else ''
    
    return {
        'longTitle': long_title,
        'shortTitle': short_title,
        'pubPlace': pub_place.text if pub_place is not None else '',
        'printer': publisher.text if publisher is not None else '',
        'date': date.text if date is not None else '',
        'type_text': type_text.text if type_text is not None else '',
        'genre': genre.text if genre is not None else '',
    }

# Dossier contenant les fichiers XML-TEI
xml_folder = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/Varios-TEI-files/xml-files'

# Ajouter les colonnes nécessaires au DataFrame
columns = ['longTitle', 'shortTitle', 'pubPlace', 'printer', 'date', 'type_text', 'genre', 'url']
for col in columns:
    xls_data[col] = ''

# Parcourir chaque ligne du DataFrame et enrichir les informations
for index, row in xls_data.iterrows():
    id_doc = row['id_doc']
    if pd.notna(id_doc):  # Vérifier si id_doc n'est pas NaN
        id_doc_str = str(id_doc)
        xml_file_name = id_doc_str.replace('Ner_', '').replace('_', '_') + '.xml'  # Conserver le soulignement
        xml_file = os.path.join(xml_folder, xml_file_name)
        
        if os.path.exists(xml_file):
            xml_info = extract_xml_info(xml_file)
            for key, value in xml_info.items():
                xls_data.at[index, key] = value
            
            # Ajouter l'URL
            xls_data.at[index, 'url'] = f'https://desenrollandoelcordel.unige.ch/Pliegos/{xml_file_name}'
        else:
            print(f'File not found: {xml_file}')
    else:
        print(f'Missing id_doc at index: {index}')

# Afficher les premières lignes du fichier XLS enrichi pour vérification
print(xls_data.head())

# Enregistrer le fichier XLS enrichi
output_file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Varios-ner/NER_Varios_wikidata_enriched_2.xlsx'
xls_data.to_excel(output_file_path, index=False, engine='openpyxl')

   Column number          id_doc   original_name       normalized_name   
0            213     Ner_BCN_025    Peñasalquera  Diócesis de Tuy-Vigo  \
1            272     Ner_BCN_029  monte Tartareo       inclasificable    
2            868  Ner_Varios_084            Muyó               El Muyo   
3            993  Ner_Varios_115          Rambla             La Rambla   
4           1053  Ner_Varios_130          Casino               Cassino   

                instancia de  Match id     longitude      latitude    id_wkd   
0       Monumentos/edificios   Q877568          42,3        8,3824   Q877568  \
1            Inclasificables       NaN           NaN           NaN       NaN   
2                  localidad  Q5822588  41.283888888  -3.317222222  Q5822588   
3  Pueblos/ciudades/regiones   Q850593     41.381389      2.173056   Q850593   
4           comuna de Italia   Q116592  41.491666666  13.833333333   Q116592   

   Column 14  Column 15  Column 16   
0        NaN        NaN        NaN  

In [15]:
#Hackathon

In [19]:
# Charger le fichier XLSX enrichi
file_path_xlsx = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/Varios-ner/NER_Varios_wikidata_enriched.xlsx'
xls_data = pd.read_excel(file_path_xlsx)

# Créer une liste pour stocker les résultats filtrés
filtered_rows = []

# Grouper les données par id_doc
grouped = xls_data.groupby('id_doc')

# Parcourir chaque groupe
for id_doc, group in grouped:
    long_title = group['longTitle'].iloc[0]  # Le titre long est le même pour chaque id_doc
    unique_locations = set()  # Pour suivre les lieux uniques ajoutés
    
    # Parcourir chaque ligne du groupe
    for index, row in group.iterrows():
        original_name = row['original_name']
        
        # Vérifier si le nom de lieu apparaît dans le titre long et n'a pas déjà été ajouté
        if pd.notna(original_name) and original_name in long_title and original_name not in unique_locations:
            unique_locations.add(original_name)  # Ajouter le lieu à l'ensemble des lieux uniques
            filtered_rows.append(row)  # Ajouter la ligne filtrée

# Convertir la liste de résultats filtrés en DataFrame
filtered_data = pd.DataFrame(filtered_rows)

# Réinitialiser l'index du DataFrame filtré
filtered_data.reset_index(drop=True, inplace=True)

# Afficher les premières lignes du fichier XLSX filtré pour vérification
print(filtered_data.head())

# Enregistrer le fichier XLSX filtré
output_file_path = '/Users/pauline/Documents/DocumentsSandoz/Sandoz/pliegos-ner/NER_Varios_wikidata_hackathon.xlsx'
filtered_data.to_excel(output_file_path, index=False, engine='openpyxl')

   Column number       id_doc original_name normalized_name   latitude   
0              1  Ner_BCN_001      Portugal        Portugal  38.700000  \
1             22  Ner_BCN_002       Sevilla         Sevilla  37.383333   
2             34  Ner_BCN_003        Toledo          Toledo  39.866667   
3             39  Ner_BCN_004         Argél           Argel  36.776389   
4             40  Ner_BCN_004     Ontiveros  Inclasificable        NaN   

   longitude           id_wkd                          type_place   
0  -9.183333              Q45  Países/reinos/imperios/continentes  \
1  -5.983333            Q8717           Pueblos/ciudades/regiones   
2  -4.033333            Q5836           Pueblos/ciudades/regiones   
3   3.058611            Q3561           Pueblos/ciudades/regiones   
4        NaN  Inclasificables                     Inclasificables   

                                           longTitle   
0  Curiosa xacara nueva, en que refiere la vida, ...  \
1  Xacara nueva, en que se r